## Manipulação de dados e prática com SQL

<h3> Incorporando todas as bases em um único arquivo </h3>

O primeiro passo do trabalho é fazer o ajuste dos dados recebidos para que todos fiquem arquivados no mesmo documento, facilitando todo o processo de envio ao MySQL.

In [1]:
import pandas as pd
import os

In [2]:
var_dir = 'C:\\Users\\lucas\\Documents\\Conhecimento\\Programming\\Data_Science\\Bootcamp_IGTI_Data_Science\\Modulo_2\\base_trabalho_pratico\\'

Aplicando o método os.listdir vai ser possível listar todos os arquivos salvos nesse caminho.

In [3]:
lista_path = os.listdir(var_dir)
lista_path[:5]

['2020-01-01.csv',
 '2020-01-02.csv',
 '2020-01-03.csv',
 '2020-01-04.csv',
 '2020-01-05.csv']

Método para juntar tudo no mesmo arquivo.

In [14]:
lista_df = []
for arquivo in lista_path:
    local_arquivo = os.path.join(var_dir, arquivo)  # faz o join dos caminhos fornecidos no for
    df = pd.read_csv(local_arquivo, sep='|', encoding = 'latin-1')
    lista_df.append(df)

In [15]:
lista_df

[   cod_pessoa data_coleta     genero data_nascimento animal_estimacao  \
 0    10141743  2020-01-01  Masculino      1981-04-25            peixe   
 1    10141860  2020-01-01   Feminino      1966-01-19            peixe   
 2    10142548  2020-01-01  Masculino      1973-08-02        tartaruga   
 3    10142591  2020-01-01  Masculino      1997-03-05             gato   
 4    10142654  2020-01-01   Feminino      1969-08-05        tartaruga   
 
       clima bebida_favorita             hobbies  
 0      frio         Cerveja    Praticar esporte  
 1    quente            Café         Assistir TV  
 2      frio            Café          Ler livros  
 3  moderado         Cerveja  Aprender algo novo  
 4  moderado            Café         Assistir TV  ,
    cod_pessoa data_coleta     genero data_nascimento animal_estimacao  \
 0    10141523  2020-01-02   Feminino      1998-09-24             gato   
 1    10141996  2020-01-02   Feminino      1973-03-13        tartaruga   
 2    10142089  2020-01-0

Uma vez que se trata de uma grande lista, é preciso converter para um dataframe. Após realizar essa etapa, será feito um export para um novo csv formatado corretamente.

In [16]:
df_trab_pratico2 = pd.concat(lista_df)  # atribui os dados coletados em um dataframe concatenado 
df_trab_pratico2.head()

,cod_pessoa,data_coleta,genero,data_nascimento,animal_estimacao,clima,bebida_favorita,hobbies
0,10141743,2020-01-01,Masculino,1981-04-25,peixe,frio,Cerveja,Praticar esporte
1,10141860,2020-01-01,Feminino,1966-01-19,peixe,quente,Café,Assistir TV
2,10142548,2020-01-01,Masculino,1973-08-02,tartaruga,frio,Café,Ler livros
3,10142591,2020-01-01,Masculino,1997-03-05,gato,moderado,Cerveja,Aprender algo novo
4,10142654,2020-01-01,Feminino,1969-08-05,tartaruga,moderado,Café,Assistir TV


In [17]:
df_trab_pratico2.to_csv(var_dir + 'trab_pratico2.csv', sep = ';', encoding = 'latin-1', index = False)

<h3> Fazendo a conexão com o banco de dados e alimentando as tabelas </h3>

Instalando as bibliotecas para manipulação do MySQL.

In [18]:
!pip install mysqlclient
!pip install sqlalchemy

Estabelecendo a conexão com o banco de dados.

In [19]:
import MySQLdb
import sqlalchemy

In [21]:
user = 'lucaslima11'
password = 'A#0B&*SsI@NmAp'
host = 'localhost'
database = 'trab_prat2'  # schema criado

string_conexao = f'mysql://{user}:{password}@{host}/{database}'

engine = sqlalchemy.create_engine(string_conexao)  # a engine usa a string que faz a conexão com o banco no MySQL
conn = engine.connect()  # aqui estabelece a conexão ao banco

Definição dos dataframes que vão alimentar as tabelas, conforme o esquema montado.

In [36]:
df_pessoa = df_trab_pratico2[['cod_pessoa','genero','data_nascimento']].drop_duplicates()
lista_bebidas = list(df_trab_pratico2.bebida_favorita.unique())
lista_hobbies = list(df_trab_pratico2.hobbies.unique())
lista_clima = list(df_trab_pratico2.clima.unique())
lista_animal = list(df_trab_pratico2.animal_estimacao.unique())

In [54]:
df_pesquisa = df_trab_pratico2[['data_coleta', 'cod_pessoa', 'animal_estimacao', 'bebida_favorita', 'hobbies', 'clima']]
df_pesquisa.reset_index(drop = True, inplace = True)
df_pesquisa.index.name = 'cod_pesquisa'
df_pesquisa.head(10)

,data_coleta,cod_pessoa,animal_estimacao,bebida_favorita,hobbies,clima
cod_pesquisa,,,,,,
0,2020-01-01,10141743,peixe,Cerveja,Praticar esporte,frio
1,2020-01-01,10141860,peixe,Café,Assistir TV,quente
2,2020-01-01,10142548,tartaruga,Café,Ler livros,frio
3,2020-01-01,10142591,gato,Cerveja,Aprender algo novo,moderado
4,2020-01-01,10142654,tartaruga,Café,Assistir TV,moderado
5,2020-01-02,10141523,gato,Cerveja,Aprender algo novo,quente
6,2020-01-02,10141996,tartaruga,refrigerante,Escrever,quente
7,2020-01-02,10142089,gato,Água,Pintar quadros,moderado
8,2020-01-02,10142145,peixe,refrigerante,Escrever,frio


Gerando as tuplas que vão alimentar as bases no MySQL.

A primeira tabela a ser alimentada vai ser a "pessoa".

In [25]:
for pessoa in df_pessoa.itertuples():
    cod_pessoa = pessoa.cod_pessoa
    genero = pessoa.genero
    data_nascimento = str(pessoa.data_nascimento)

    try:
        query = f"""insert into pessoa (cod_pessoa, genero,  data_nascimento) 
                     values ('{cod_pessoa}', '{genero}',  '{data_nascimento}')"""      
        conn.execute(query)      

        print(f"Registro inserido com sucesso: {cod_pessoa}")

    except Exception as e:        
        print(f"Não foi possível inserir o registro {cod_pessoa}. O erro foi encontrado foi: {e}")    

Registro inserido com sucesso: 10141743
Registro inserido com sucesso: 10141860
Registro inserido com sucesso: 10142548
Registro inserido com sucesso: 10142591
Registro inserido com sucesso: 10142654
Registro inserido com sucesso: 10141523
Registro inserido com sucesso: 10141996
Registro inserido com sucesso: 10142089
Registro inserido com sucesso: 10142145
Registro inserido com sucesso: 10142520
Registro inserido com sucesso: 10143160
Registro inserido com sucesso: 10141712
Registro inserido com sucesso: 10141987
Registro inserido com sucesso: 10142093
Registro inserido com sucesso: 10142325
Registro inserido com sucesso: 10142552
Registro inserido com sucesso: 10142598
Registro inserido com sucesso: 10143013
Registro inserido com sucesso: 10141570
Registro inserido com sucesso: 10141851
Registro inserido com sucesso: 10141938
Registro inserido com sucesso: 10142733
Registro inserido com sucesso: 10142851
Registro inserido com sucesso: 10142941
Registro inserido com sucesso: 10143484


Registro inserido com sucesso: 10143131
Registro inserido com sucesso: 10141587
Registro inserido com sucesso: 10141630
Registro inserido com sucesso: 10142002
Registro inserido com sucesso: 10142503
Registro inserido com sucesso: 10142625
Registro inserido com sucesso: 10142961
Registro inserido com sucesso: 10143252
Registro inserido com sucesso: 10143356
Registro inserido com sucesso: 10143400
Registro inserido com sucesso: 10143512
Registro inserido com sucesso: 10141681
Registro inserido com sucesso: 10142581
Registro inserido com sucesso: 10142736
Registro inserido com sucesso: 10142839
Registro inserido com sucesso: 10143027
Registro inserido com sucesso: 10143048
Registro inserido com sucesso: 10141526
Registro inserido com sucesso: 10141605
Registro inserido com sucesso: 10141609
Registro inserido com sucesso: 10142249
Registro inserido com sucesso: 10142282
Registro inserido com sucesso: 10142602
Registro inserido com sucesso: 10143026
Registro inserido com sucesso: 10143292


Registro inserido com sucesso: 10142704
Registro inserido com sucesso: 10142738
Registro inserido com sucesso: 10142933
Registro inserido com sucesso: 10143010
Registro inserido com sucesso: 10143088
Registro inserido com sucesso: 10143189
Registro inserido com sucesso: 10143455
Registro inserido com sucesso: 10141903
Registro inserido com sucesso: 10142496
Registro inserido com sucesso: 10142633
Registro inserido com sucesso: 10142662
Registro inserido com sucesso: 10142841
Registro inserido com sucesso: 10143076
Registro inserido com sucesso: 10143617
Registro inserido com sucesso: 10141585
Registro inserido com sucesso: 10141825
Registro inserido com sucesso: 10142067
Registro inserido com sucesso: 10142797
Registro inserido com sucesso: 10142920
Registro inserido com sucesso: 10143090
Registro inserido com sucesso: 10143385
Registro inserido com sucesso: 10143441
Registro inserido com sucesso: 10141723
Registro inserido com sucesso: 10141936
Registro inserido com sucesso: 10142075


Registro inserido com sucesso: 10143224
Registro inserido com sucesso: 10143375
Registro inserido com sucesso: 10142108
Registro inserido com sucesso: 10142332
Registro inserido com sucesso: 10142696
Registro inserido com sucesso: 10142810
Registro inserido com sucesso: 10143011
Registro inserido com sucesso: 10143203
Registro inserido com sucesso: 10142115
Registro inserido com sucesso: 10142749
Registro inserido com sucesso: 10142972
Registro inserido com sucesso: 10143610
Registro inserido com sucesso: 10142296
Registro inserido com sucesso: 10142709
Registro inserido com sucesso: 10143578
Registro inserido com sucesso: 10142188
Registro inserido com sucesso: 10142197
Registro inserido com sucesso: 10142528
Registro inserido com sucesso: 10142566
Registro inserido com sucesso: 10143335
Registro inserido com sucesso: 10143392
Registro inserido com sucesso: 10143471
Registro inserido com sucesso: 10143563
Registro inserido com sucesso: 10141537
Registro inserido com sucesso: 10142913


Registro inserido com sucesso: 10143243
Registro inserido com sucesso: 10143332
Registro inserido com sucesso: 10143540
Registro inserido com sucesso: 10143576
Registro inserido com sucesso: 10141507
Registro inserido com sucesso: 10142152
Registro inserido com sucesso: 10142288
Registro inserido com sucesso: 10142943
Registro inserido com sucesso: 10143259
Registro inserido com sucesso: 10143408
Registro inserido com sucesso: 10142144
Registro inserido com sucesso: 10142624
Registro inserido com sucesso: 10142942
Registro inserido com sucesso: 10143263
Registro inserido com sucesso: 10142016
Registro inserido com sucesso: 10142351
Registro inserido com sucesso: 10142634
Registro inserido com sucesso: 10143238
Registro inserido com sucesso: 10141797
Registro inserido com sucesso: 10142066
Registro inserido com sucesso: 10142091
Registro inserido com sucesso: 10142340
Registro inserido com sucesso: 10142461
Registro inserido com sucesso: 10143348
Registro inserido com sucesso: 10143391


Registro inserido com sucesso: 10142178
Registro inserido com sucesso: 10143005
Registro inserido com sucesso: 10143564
Registro inserido com sucesso: 10142037
Registro inserido com sucesso: 10142592
Registro inserido com sucesso: 10143378
Registro inserido com sucesso: 10143442
Registro inserido com sucesso: 10142097
Registro inserido com sucesso: 10142701
Registro inserido com sucesso: 10142823
Registro inserido com sucesso: 10143360
Registro inserido com sucesso: 10143364
Registro inserido com sucesso: 10143434
Registro inserido com sucesso: 10141624
Registro inserido com sucesso: 10141638
Registro inserido com sucesso: 10141706
Registro inserido com sucesso: 10141795
Registro inserido com sucesso: 10141838
Registro inserido com sucesso: 10142189
Registro inserido com sucesso: 10143065
Registro inserido com sucesso: 10143205
Registro inserido com sucesso: 10141857
Registro inserido com sucesso: 10141930
Registro inserido com sucesso: 10142285
Registro inserido com sucesso: 10143222


Registro inserido com sucesso: 10142389
Registro inserido com sucesso: 10142752
Registro inserido com sucesso: 10143022
Registro inserido com sucesso: 10143117
Registro inserido com sucesso: 10143456
Registro inserido com sucesso: 10143544
Registro inserido com sucesso: 10142529
Registro inserido com sucesso: 10142799
Registro inserido com sucesso: 10143015
Registro inserido com sucesso: 10143447
Registro inserido com sucesso: 10143495
Registro inserido com sucesso: 10141782
Registro inserido com sucesso: 10141946
Registro inserido com sucesso: 10142517
Registro inserido com sucesso: 10142895
Registro inserido com sucesso: 10143379
Registro inserido com sucesso: 10143618
Registro inserido com sucesso: 10141717
Registro inserido com sucesso: 10141918
Registro inserido com sucesso: 10142237
Registro inserido com sucesso: 10142320
Registro inserido com sucesso: 10142500
Registro inserido com sucesso: 10142585
Registro inserido com sucesso: 10142914
Registro inserido com sucesso: 10142929


Registro inserido com sucesso: 10143517
Registro inserido com sucesso: 10141623
Registro inserido com sucesso: 10141974
Registro inserido com sucesso: 10142213
Registro inserido com sucesso: 10142457
Registro inserido com sucesso: 10142490
Registro inserido com sucesso: 10143083
Registro inserido com sucesso: 10143254
Registro inserido com sucesso: 10143479
Registro inserido com sucesso: 10141725
Registro inserido com sucesso: 10141731
Registro inserido com sucesso: 10141800
Registro inserido com sucesso: 10141852
Registro inserido com sucesso: 10142045
Registro inserido com sucesso: 10142147
Registro inserido com sucesso: 10142244
Registro inserido com sucesso: 10142331
Registro inserido com sucesso: 10143042
Registro inserido com sucesso: 10143215
Registro inserido com sucesso: 10141719
Registro inserido com sucesso: 10141868
Registro inserido com sucesso: 10141883
Registro inserido com sucesso: 10142375
Registro inserido com sucesso: 10142957
Registro inserido com sucesso: 10143528


Registro inserido com sucesso: 10141804
Registro inserido com sucesso: 10141978
Registro inserido com sucesso: 10142122
Registro inserido com sucesso: 10142590
Registro inserido com sucesso: 10142643
Registro inserido com sucesso: 10143066
Registro inserido com sucesso: 10142402
Registro inserido com sucesso: 10142440
Registro inserido com sucesso: 10142775
Registro inserido com sucesso: 10142801
Registro inserido com sucesso: 10143165
Registro inserido com sucesso: 10141986
Registro inserido com sucesso: 10142442
Registro inserido com sucesso: 10142922
Registro inserido com sucesso: 10143030
Registro inserido com sucesso: 10143138
Registro inserido com sucesso: 10143217
Registro inserido com sucesso: 10143311
Registro inserido com sucesso: 10143419
Registro inserido com sucesso: 10141558
Registro inserido com sucesso: 10141567
Registro inserido com sucesso: 10141604
Registro inserido com sucesso: 10141921
Registro inserido com sucesso: 10141976
Registro inserido com sucesso: 10143388


In [28]:
df_pessoa.count()

cod_pessoa         2127
genero             2127
data_nascimento    2127
dtype: int64

As próximas tabelas com valores únicos serão alimentadas.

In [32]:
for bebida in lista_bebidas:
    try:
        query = f"""insert into bebida (bebida) 
                     values ('{bebida}')"""      
        conn.execute(query)      

        print(f"Registro inserido com sucesso: {bebida}")

    except Exception as e:        
        print(f"Não foi possível inserir o registro {bebida}. O erro foi encontrado foi: {e}")    

Registro inserido com sucesso: Cerveja
Registro inserido com sucesso: Café
Registro inserido com sucesso: refrigerante
Registro inserido com sucesso: Água
Registro inserido com sucesso: Vinho
Registro inserido com sucesso: Chá


In [33]:
df_bebida = pd.read_sql('bebida', con = conn)
df_bebida

,cod_bebida,bebida
0,4,Água
1,2,Café
2,1,Cerveja
3,6,Chá
4,3,refrigerante
5,5,Vinho


In [38]:
for hobby in lista_hobbies:
    try:
        query = f"""insert into hobbie (hobbie) 
                     values ('{hobby}')"""      
        conn.execute(query)      

        print(f"Registro inserido com sucesso: {hobby}")

    except Exception as e:        
        print(f"Não foi possível inserir o registro {hobby}. O erro foi encontrado foi: {e}")    

Registro inserido com sucesso: Praticar esporte
Registro inserido com sucesso: Assistir TV
Registro inserido com sucesso: Ler livros
Registro inserido com sucesso: Aprender algo novo
Registro inserido com sucesso: Escrever
Registro inserido com sucesso: Pintar quadros
Registro inserido com sucesso: Dormir
Registro inserido com sucesso: Escutar música


In [60]:
df_hobbies = pd.read_sql('hobbie', con = conn)
df_hobbies.head()

,cod_hobbie,hobbie
0,4,Aprender algo novo
1,2,Assistir TV
2,7,Dormir
3,5,Escrever
4,8,Escutar música


In [39]:
for clima in lista_clima:
    try:
        query = f"""insert into clima (clima) 
                     values ('{clima}')"""      
        conn.execute(query)      

        print(f"Registro inserido com sucesso: {clima}")

    except Exception as e:        
        print(f"Não foi possível inserir o registro {clima}. O erro foi encontrado foi: {e}")  

Registro inserido com sucesso: frio
Registro inserido com sucesso: quente
Registro inserido com sucesso: moderado


In [61]:
df_clima = pd.read_sql('clima', con = conn)
df_clima.head()

,cod_clima,clima
0,1,frio
1,2,quente
2,3,moderado


In [41]:
for animal in lista_animal:
    try:
        query = f"""insert into animal_estimacao (animal) 
                     values ('{animal}')"""      
        conn.execute(query)      

        print(f"Registro inserido com sucesso: {animal}")

    except Exception as e:        
        print(f"Não foi possível inserir o registro {animal}. O erro foi encontrado foi: {e}")  

Registro inserido com sucesso: peixe
Registro inserido com sucesso: tartaruga
Registro inserido com sucesso: gato
Registro inserido com sucesso: cachorro


In [62]:
df_animal = pd.read_sql('animal_estimacao', con = conn)
df_animal.head()

,cod_animal_estimacao,animal
0,4,cachorro
1,3,gato
2,1,peixe
3,2,tartaruga


Para alimentar a tabela que concatena todas as chaves, cria-se uma tabela adicional no banco incluindo toda a base, para que, dentro do MySQL, se utiliza um método para selecionar as colunas desejadas. Esse método também é conhecido como staging.

In [42]:
df_trab_pratico2.to_sql('stg_base_pesquisa', con = conn, schema = 'trab_prat2') #gerou a base, mas não vou aplicar esse método

Aplicando o método de definição de dicionário para as colunas que se vinculam à base principal, aplicarei todas essas informações, associado à criação de um index como fonte para a coluna cod_pesquisa.

In [63]:
dict_bebida = df_bebida.set_index(df_bebida.bebida)['cod_bebida'].to_dict()
dict_bebida

{'Água': 4, 'Café': 2, 'Cerveja': 1, 'Chá': 6, 'refrigerante': 3, 'Vinho': 5}

In [64]:
dict_hobbies = df_hobbies.set_index(df_hobbies.hobbie)['cod_hobbie'].to_dict()
dict_hobbies

{'Aprender algo novo': 4,
 'Assistir TV': 2,
 'Dormir': 7,
 'Escrever': 5,
 'Escutar música': 8,
 'Ler livros': 3,
 'Pintar quadros': 6,
 'Praticar esporte': 1}

In [65]:
dict_clima = df_clima.set_index(df_clima.clima)['cod_clima'].to_dict()
dict_clima

{'frio': 1, 'quente': 2, 'moderado': 3}

In [66]:
dict_animal = df_animal.set_index(df_animal.animal)['cod_animal_estimacao'].to_dict()
dict_animal

{'cachorro': 4, 'gato': 3, 'peixe': 1, 'tartaruga': 2}

In [67]:
# df_pesquisa = df_trab_pratico2[['data_coleta', 'cod_pessoa', 'animal_estimacao', 'bebida_favorita', 'hobbies', 'clima']]
df_pesquisa.animal_estimacao = df_pesquisa.animal_estimacao.map(dict_animal)
df_pesquisa.bebida_favorita = df_pesquisa.bebida_favorita.map(dict_bebida) 
df_pesquisa.hobbies = df_pesquisa.hobbies.map(dict_hobbies) 
df_pesquisa.clima = df_pesquisa.clima.map(dict_clima)
df_pesquisa.head()

c:\users\lucas\documents\conhecimento\programming\data_science\bootcamp_igti_data_science\modulo_2\trab_pratico2\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,data_coleta,cod_pessoa,animal_estimacao,bebida_favorita,hobbies,clima
cod_pesquisa,,,,,,
0,2020-01-01,10141743,1,1,1,1
1,2020-01-01,10141860,1,2,2,2
2,2020-01-01,10142548,2,2,3,1
3,2020-01-01,10142591,3,1,4,3
4,2020-01-01,10142654,2,2,2,3


Renomeando as colunas

In [68]:
df_pesquisa = df_pesquisa.rename(columns={'animal_estimacao' : 'cod_animal_estimacao'})
df_pesquisa = df_pesquisa.rename(columns={'bebida_favorita' : 'cod_bebida'})
df_pesquisa = df_pesquisa.rename(columns={'hobbies' : 'cod_hobbie'})
df_pesquisa = df_pesquisa.rename(columns={'clima' : 'cod_clima'})
df_pesquisa = df_pesquisa.rename(columns={'data_coleta' : 'data_pesquisa'})
df_pesquisa

,data_pesquisa,cod_pessoa,cod_animal_estimacao,cod_bebida,cod_hobbie,cod_clima
cod_pesquisa,,,,,,
0,2020-01-01,10141743,1,1,1,1
1,2020-01-01,10141860,1,2,2,2
2,2020-01-01,10142548,2,2,3,1
3,2020-01-01,10142591,3,1,4,3
4,2020-01-01,10142654,2,2,2,3
...,...,...,...,...,...,...
2122,2020-12-31,10142475,1,2,1,2
2123,2020-12-31,10142522,1,1,1,3
2124,2020-12-31,10142881,3,3,2,1


Subindo a base para o MySQL.

In [82]:
for pesquisa in df_pesquisa.itertuples():
    data_pesquisa = pesquisa.data_pesquisa
    cod_animal_estimacao = pesquisa.cod_animal_estimacao
    cod_bebida = pesquisa.cod_bebida
    cod_clima = pesquisa.cod_clima
    cod_hobbie = pesquisa.cod_hobbie
    cod_pessoa = pesquisa.cod_pessoa
    
    try:
        query = f"""insert into pesquisa (data_pesquisa, cod_animal_estimacao, cod_bebida, cod_clima, cod_hobbie, cod_pessoa) 
                     values ('{data_pesquisa}', '{cod_animal_estimacao}',  '{cod_bebida}', '{cod_clima}', '{cod_hobbie}', '{cod_pessoa}')"""      
        conn.execute(query)      

        print(f"Registro inserido com sucesso: {data_pesquisa}")

    except Exception as e:        
        print(f"Não foi possível inserir o registro {data_pesquisa}. O erro foi encontrado foi: {e}") 

Registro inserido com sucesso: 2020-01-01
Registro inserido com sucesso: 2020-01-01
Registro inserido com sucesso: 2020-01-01
Registro inserido com sucesso: 2020-01-01
Registro inserido com sucesso: 2020-01-01
Registro inserido com sucesso: 2020-01-02
Registro inserido com sucesso: 2020-01-02
Registro inserido com sucesso: 2020-01-02
Registro inserido com sucesso: 2020-01-02
Registro inserido com sucesso: 2020-01-02
Registro inserido com sucesso: 2020-01-02
Registro inserido com sucesso: 2020-01-03
Registro inserido com sucesso: 2020-01-03
Registro inserido com sucesso: 2020-01-03
Registro inserido com sucesso: 2020-01-03
Registro inserido com sucesso: 2020-01-03
Registro inserido com sucesso: 2020-01-03
Registro inserido com sucesso: 2020-01-03
Registro inserido com sucesso: 2020-01-04
Registro inserido com sucesso: 2020-01-04
Registro inserido com sucesso: 2020-01-04
Registro inserido com sucesso: 2020-01-04
Registro inserido com sucesso: 2020-01-04
Registro inserido com sucesso: 202

Registro inserido com sucesso: 2020-02-03
Registro inserido com sucesso: 2020-02-03
Registro inserido com sucesso: 2020-02-03
Registro inserido com sucesso: 2020-02-03
Registro inserido com sucesso: 2020-02-03
Registro inserido com sucesso: 2020-02-03
Registro inserido com sucesso: 2020-02-04
Registro inserido com sucesso: 2020-02-04
Registro inserido com sucesso: 2020-02-04
Registro inserido com sucesso: 2020-02-04
Registro inserido com sucesso: 2020-02-04
Registro inserido com sucesso: 2020-02-04
Registro inserido com sucesso: 2020-02-04
Registro inserido com sucesso: 2020-02-05
Registro inserido com sucesso: 2020-02-05
Registro inserido com sucesso: 2020-02-05
Registro inserido com sucesso: 2020-02-05
Registro inserido com sucesso: 2020-02-05
Registro inserido com sucesso: 2020-02-05
Registro inserido com sucesso: 2020-02-05
Registro inserido com sucesso: 2020-02-06
Registro inserido com sucesso: 2020-02-06
Registro inserido com sucesso: 2020-02-06
Registro inserido com sucesso: 202

Registro inserido com sucesso: 2020-03-14
Registro inserido com sucesso: 2020-03-14
Registro inserido com sucesso: 2020-03-14
Registro inserido com sucesso: 2020-03-14
Registro inserido com sucesso: 2020-03-14
Registro inserido com sucesso: 2020-03-14
Registro inserido com sucesso: 2020-03-14
Registro inserido com sucesso: 2020-03-15
Registro inserido com sucesso: 2020-03-15
Registro inserido com sucesso: 2020-03-15
Registro inserido com sucesso: 2020-03-15
Registro inserido com sucesso: 2020-03-15
Registro inserido com sucesso: 2020-03-15
Registro inserido com sucesso: 2020-03-15
Registro inserido com sucesso: 2020-03-16
Registro inserido com sucesso: 2020-03-16
Registro inserido com sucesso: 2020-03-17
Registro inserido com sucesso: 2020-03-17
Registro inserido com sucesso: 2020-03-17
Registro inserido com sucesso: 2020-03-17
Registro inserido com sucesso: 2020-03-17
Registro inserido com sucesso: 2020-03-18
Registro inserido com sucesso: 2020-03-18
Registro inserido com sucesso: 202

Registro inserido com sucesso: 2020-05-02
Registro inserido com sucesso: 2020-05-02
Registro inserido com sucesso: 2020-05-03
Registro inserido com sucesso: 2020-05-03
Registro inserido com sucesso: 2020-05-03
Registro inserido com sucesso: 2020-05-03
Registro inserido com sucesso: 2020-05-04
Registro inserido com sucesso: 2020-05-04
Registro inserido com sucesso: 2020-05-04
Registro inserido com sucesso: 2020-05-04
Registro inserido com sucesso: 2020-05-05
Registro inserido com sucesso: 2020-05-05
Registro inserido com sucesso: 2020-05-05
Registro inserido com sucesso: 2020-05-05
Registro inserido com sucesso: 2020-05-05
Registro inserido com sucesso: 2020-05-05
Registro inserido com sucesso: 2020-05-06
Registro inserido com sucesso: 2020-05-06
Registro inserido com sucesso: 2020-05-06
Registro inserido com sucesso: 2020-05-06
Registro inserido com sucesso: 2020-05-06
Registro inserido com sucesso: 2020-05-06
Registro inserido com sucesso: 2020-05-06
Registro inserido com sucesso: 202

Registro inserido com sucesso: 2020-06-21
Registro inserido com sucesso: 2020-06-21
Registro inserido com sucesso: 2020-06-21
Registro inserido com sucesso: 2020-06-22
Registro inserido com sucesso: 2020-06-22
Registro inserido com sucesso: 2020-06-22
Registro inserido com sucesso: 2020-06-22
Registro inserido com sucesso: 2020-06-22
Registro inserido com sucesso: 2020-06-22
Registro inserido com sucesso: 2020-06-22
Registro inserido com sucesso: 2020-06-23
Registro inserido com sucesso: 2020-06-23
Registro inserido com sucesso: 2020-06-23
Registro inserido com sucesso: 2020-06-23
Registro inserido com sucesso: 2020-06-24
Registro inserido com sucesso: 2020-06-24
Registro inserido com sucesso: 2020-06-24
Registro inserido com sucesso: 2020-06-24
Registro inserido com sucesso: 2020-06-24
Registro inserido com sucesso: 2020-06-24
Registro inserido com sucesso: 2020-06-24
Registro inserido com sucesso: 2020-06-24
Registro inserido com sucesso: 2020-06-24
Registro inserido com sucesso: 202

Registro inserido com sucesso: 2020-08-11
Registro inserido com sucesso: 2020-08-11
Registro inserido com sucesso: 2020-08-11
Registro inserido com sucesso: 2020-08-11
Registro inserido com sucesso: 2020-08-11
Registro inserido com sucesso: 2020-08-11
Registro inserido com sucesso: 2020-08-12
Registro inserido com sucesso: 2020-08-12
Registro inserido com sucesso: 2020-08-12
Registro inserido com sucesso: 2020-08-12
Registro inserido com sucesso: 2020-08-12
Registro inserido com sucesso: 2020-08-12
Registro inserido com sucesso: 2020-08-12
Registro inserido com sucesso: 2020-08-12
Registro inserido com sucesso: 2020-08-13
Registro inserido com sucesso: 2020-08-13
Registro inserido com sucesso: 2020-08-13
Registro inserido com sucesso: 2020-08-13
Registro inserido com sucesso: 2020-08-14
Registro inserido com sucesso: 2020-08-14
Registro inserido com sucesso: 2020-08-14
Registro inserido com sucesso: 2020-08-15
Registro inserido com sucesso: 2020-08-15
Registro inserido com sucesso: 202

Registro inserido com sucesso: 2020-09-27
Registro inserido com sucesso: 2020-09-27
Registro inserido com sucesso: 2020-09-27
Registro inserido com sucesso: 2020-09-27
Registro inserido com sucesso: 2020-09-28
Registro inserido com sucesso: 2020-09-28
Registro inserido com sucesso: 2020-09-28
Registro inserido com sucesso: 2020-09-28
Registro inserido com sucesso: 2020-09-28
Registro inserido com sucesso: 2020-09-28
Registro inserido com sucesso: 2020-09-29
Registro inserido com sucesso: 2020-09-30
Registro inserido com sucesso: 2020-09-30
Registro inserido com sucesso: 2020-09-30
Registro inserido com sucesso: 2020-09-30
Registro inserido com sucesso: 2020-09-30
Registro inserido com sucesso: 2020-09-30
Registro inserido com sucesso: 2020-10-02
Registro inserido com sucesso: 2020-10-02
Registro inserido com sucesso: 2020-10-02
Registro inserido com sucesso: 2020-10-02
Registro inserido com sucesso: 2020-10-02
Registro inserido com sucesso: 2020-10-02
Registro inserido com sucesso: 202

Registro inserido com sucesso: 2020-11-10
Registro inserido com sucesso: 2020-11-10
Registro inserido com sucesso: 2020-11-10
Registro inserido com sucesso: 2020-11-11
Registro inserido com sucesso: 2020-11-11
Registro inserido com sucesso: 2020-11-11
Registro inserido com sucesso: 2020-11-11
Registro inserido com sucesso: 2020-11-11
Registro inserido com sucesso: 2020-11-11
Registro inserido com sucesso: 2020-11-11
Registro inserido com sucesso: 2020-11-11
Registro inserido com sucesso: 2020-11-11
Registro inserido com sucesso: 2020-11-12
Registro inserido com sucesso: 2020-11-12
Registro inserido com sucesso: 2020-11-12
Registro inserido com sucesso: 2020-11-12
Registro inserido com sucesso: 2020-11-12
Registro inserido com sucesso: 2020-11-12
Registro inserido com sucesso: 2020-11-12
Registro inserido com sucesso: 2020-11-12
Registro inserido com sucesso: 2020-11-13
Registro inserido com sucesso: 2020-11-13
Registro inserido com sucesso: 2020-11-13
Registro inserido com sucesso: 202

Registro inserido com sucesso: 2020-12-22
Registro inserido com sucesso: 2020-12-22
Registro inserido com sucesso: 2020-12-22
Registro inserido com sucesso: 2020-12-23
Registro inserido com sucesso: 2020-12-23
Registro inserido com sucesso: 2020-12-23
Registro inserido com sucesso: 2020-12-23
Registro inserido com sucesso: 2020-12-23
Registro inserido com sucesso: 2020-12-24
Registro inserido com sucesso: 2020-12-24
Registro inserido com sucesso: 2020-12-24
Registro inserido com sucesso: 2020-12-24
Registro inserido com sucesso: 2020-12-24
Registro inserido com sucesso: 2020-12-24
Registro inserido com sucesso: 2020-12-24
Registro inserido com sucesso: 2020-12-24
Registro inserido com sucesso: 2020-12-24
Registro inserido com sucesso: 2020-12-25
Registro inserido com sucesso: 2020-12-25
Registro inserido com sucesso: 2020-12-25
Registro inserido com sucesso: 2020-12-26
Registro inserido com sucesso: 2020-12-26
Registro inserido com sucesso: 2020-12-26
Registro inserido com sucesso: 202

In [83]:
df_pesquisa_cons = pd.read_sql('pesquisa', con = conn)
df_pesquisa_cons

,cod_pesquisa,data_pesquisa,cod_animal_estimacao,cod_bebida,cod_clima,cod_hobbie,cod_pessoa
0,1,2020-01-01,1,1,1,1,10141743
1,2,2020-01-01,1,2,2,2,10141860
2,3,2020-01-01,2,2,1,3,10142548
3,4,2020-01-01,3,1,3,4,10142591
4,5,2020-01-01,2,2,3,2,10142654
...,...,...,...,...,...,...,...
2122,2123,2020-12-31,1,2,2,1,10142475
2123,2124,2020-12-31,1,1,3,1,10142522
2124,2125,2020-12-31,3,3,1,2,10142881
2125,2126,2020-12-31,2,3,2,6,10143184
